In [1]:
# import modules
import pandas as pd
from tensorflow.keras import metrics
import tensorflow.keras.backend as K
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
import numpy as np
import os
from skimage import io
from tqdm import tqdm

In [2]:
# # set GPU
# gpus = tf.config.experimental.list_physical_devices('GPU')
# tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
# for gpu in gpus:
#     tf.config.experimental.set_memory_growth(gpu, True)
# print(len(gpus))
# logical_gpus = tf.config.experimental.list_logical_devices('GPU')
# print(len(logical_gpus))

In [3]:
# read data
jan_train = pd.read_csv('training1.csv')

In [4]:
jan_train_x = pd.concat([jan_train.iloc[:,5:12]/jan_train.iloc[:,5:12].max(), jan_train.iloc[:,3:5]/jan_train.iloc[:,3:5].max(),
                     jan_train.iloc[:,15:16]/jan_train.iloc[:,15:16].max(), jan_train.iloc[:,13:14]/jan_train.iloc[:,13:14].max(),
                     jan_train.iloc[:,16:17]/jan_train.iloc[:,16:17].max(), jan_train.iloc[:,14:15]/jan_train.iloc[:,14:15].max(),
                     jan_train.iloc[:,2:3]/jan_train.iloc[:,2:3].max(), jan_train.iloc[:,1:2]/jan_train.iloc[:,1:2].max(),
                     jan_train.iloc[:,12:13]/jan_train.iloc[:,12:13].max()],axis=1,join='inner').values
jan_train_y = jan_train.landcover.values

In [5]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(jan_train_x, jan_train_y, test_size=0.2, random_state=3)
print(len(X_train), len(X_test))

320 80


In [6]:
# set training model
logdir = os.path.join("model")
if not os.path.exists(logdir):
    os.mkdir(logdir)
output_model_file = os.path.join(logdir,"model.h5")

callbacks = [
    keras.callbacks.TensorBoard(logdir),
    keras.callbacks.ModelCheckpoint(output_model_file,monitor='val_accuracy',save_best_only = True,save_weights_only = False),
    keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=100, min_delta=1e-7),
]

In [7]:
# DNN structure
model = Sequential()
model.add(Dense(32, input_shape=(16,)))
model.add(Activation('relu'))
model.add(Dropout(0.1))
model.add(Dense(16))
model.add(Activation('relu'))
model.add(Dropout(0.1))
model.add(Dense(6))
model.add(Activation('softmax'))
# set loss and optimizer
model.compile(loss="sparse_categorical_crossentropy",optimizer = tf.keras.optimizers.Adam(),metrics = ["accuracy"])
# train the model
model.fit(X_train, Y_train, epochs=100, batch_size=64,shuffle=True,
          validation_data=(X_test,Y_test),callbacks=callbacks,verbose=1)

Train on 320 samples, validate on 80 samples
Epoch 1/100
320/320 [==============================] - 1s 2ms/sample - loss: 1.8602 - accuracy: 0.2125 - val_loss: 1.8374 - val_accuracy: 0.1500
Epoch 2/100
320/320 [==============================] - 0s 103us/sample - loss: 1.7944 - accuracy: 0.2062 - val_loss: 1.7970 - val_accuracy: 0.1500
Epoch 3/100
320/320 [==============================] - 0s 162us/sample - loss: 1.7579 - accuracy: 0.2281 - val_loss: 1.7593 - val_accuracy: 0.1750
Epoch 4/100
320/320 [==============================] - 0s 167us/sample - loss: 1.7185 - accuracy: 0.2906 - val_loss: 1.7238 - val_accuracy: 0.2500
Epoch 5/100
320/320 [==============================] - 0s 155us/sample - loss: 1.6886 - accuracy: 0.3281 - val_loss: 1.6896 - val_accuracy: 0.2750
Epoch 6/100
320/320 [==============================] - 0s 158us/sample - loss: 1.6415 - accuracy: 0.3531 - val_loss: 1.6549 - val_accuracy: 0.3000
Epoch 7/100
320/320 [==============================] - 0s 90us/sample - los

320/320 [==============================] - 0s 84us/sample - loss: 0.7376 - accuracy: 0.6969 - val_loss: 0.7363 - val_accuracy: 0.6625
Epoch 57/100
320/320 [==============================] - 0s 87us/sample - loss: 0.7235 - accuracy: 0.7125 - val_loss: 0.7343 - val_accuracy: 0.6500
Epoch 58/100
320/320 [==============================] - 0s 82us/sample - loss: 0.7162 - accuracy: 0.7031 - val_loss: 0.7307 - val_accuracy: 0.6500
Epoch 59/100
320/320 [==============================] - 0s 84us/sample - loss: 0.7111 - accuracy: 0.6875 - val_loss: 0.7264 - val_accuracy: 0.6500
Epoch 60/100
320/320 [==============================] - 0s 80us/sample - loss: 0.6849 - accuracy: 0.7156 - val_loss: 0.7237 - val_accuracy: 0.6625
Epoch 61/100
320/320 [==============================] - ETA: 0s - loss: 0.6625 - accuracy: 0.76 - 0s 80us/sample - loss: 0.7134 - accuracy: 0.6906 - val_loss: 0.7217 - val_accuracy: 0.6625
Epoch 62/100
320/320 [==============================] - 0s 90us/sample - loss: 0.7074 - a

In [8]:
model = keras.models.load_model(output_model_file)

In [9]:
# read image to be classified
jan_img = np.reshape(io.imread('JanuaryClip.tif'),(-1,13))

In [10]:
h,w,c = io.imread('JanuaryClip.tif').shape
jan_img_b2_b8 = np.concatenate((jan_img[:,1:8], jan_img[:,11:13]), axis=1)

In [11]:
# geoTIFF image pre process
ndvi = (jan_img[:,7]-jan_img[:,3])/(jan_img[:,7]+jan_img[:,3])
ndbi = (jan_img[:,1]-jan_img[:,7])/(jan_img[:,1]+jan_img[:,7])
ndwi = (jan_img[:,2]-jan_img[:,7])/(jan_img[:,2]+jan_img[:,7])
ndsi = (jan_img[:,2]-jan_img[:,11])/(jan_img[:,2]+jan_img[:,11])
aweis = jan_img[:,1]+2.5*jan_img[:,2]-1.5*(jan_img[:,7]+jan_img[:,11])-0.25*jan_img[:,12]
aweins = 4*(jan_img[:,2]-jan_img[:,11])-(0.25*jan_img[:,7]+2.75*jan_img[:,12])
evi = 2.5*(jan_img[:,7]-jan_img[:,3])/(jan_img[:,7]+6*jan_img[:,3]-7.5*jan_img[:,1]+1)
divd = np.concatenate((jan_train.iloc[:,5:12].max().values, jan_train.iloc[:,3:5].max().values, \
                     jan_train.iloc[:,15:16].max().values, jan_train.iloc[:,13:14].max().values, \
                     jan_train.iloc[:,16:17].max().values, jan_train.iloc[:,14:15].max().values, \
                     jan_train.iloc[:,2:3].max().values, jan_train.iloc[:,1:2].max().values, \
                     jan_train.iloc[:,12:13].max().values))
jan_test_data = np.c_[jan_img_b2_b8,ndvi,ndbi,ndwi,ndsi,aweis,aweins,evi]/divd

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in true_divide
  
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in true_divide
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in true_divide
  after removing the cwd from sys.path.
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in true_divide
  """
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: RuntimeWarning: divide by zero encountered in true_divide
  
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in true_divide
  


In [ ]:
# predict the class
pred = []
invalid_data = []
valid_data = []
for i in tqdm(range(len(jan_img))):
    if jan_test_data[i][0] == 0 or jan_test_data[i][0] == -1:
        pred.append(-1)
    else:
        pred.append(model.predict(jan_test_data[i][np.newaxis,::]).argmax())

  2%|█▍                                                                     | 55723/2756160 [38:37<22:18:22, 33.63it/s]